<h1>Experiment 1</h1>
<h3>Test the number of layers acquired to output promising NODDI measure for three networks</h3>

In [ ]:
import nibabel as nib
import numpy as np

from FormatData import generate_data, parser as data_parser
from Training import train_network
from utils.model import parser as model_parser
from Testing import test_model
from utils.nii_utils import mask_nii_data

In [37]:
train_subject = 's01_still'
# test_subjects = ['s02_still', 's03_still_reg', 's04_still_reg']
test_subjects = ['s02_still']

In [ ]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [ ]:
"""
packages that handle graphs
"""
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from skimage.metrics import structural_similarity as compare_ssim
from utils import calc_ssim
%matplotlib inline
def plot_loss(cmd):
    """
    A function that used to plot the loss curve for the trained network.
    Args:
        cmd: String, the command line in the terminal
    """
    args = model_parser().parse_args(cmd.split())
    history = train_network(args)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

def show_slices(slices, grayscale=True):
    """
    Function to display the slices

    Args:
        slices (list): a list of 2d ndarray that contains the data to be displayed
        grayscale (bool, optional): True, if diplay grayscale img. Defaults to True.
    """    
    fig, axes = plt.subplots(1, len(slices), figsize=(10,10))
    cax = fig.add_axes([0, 0, .3, .3])
    for i, slice in enumerate(slices):
        # use grayscale for displaying ref and pred imgs:
        if grayscale:
            cmap = plt.get_cmap('gray')
            cmaplist = [cmap(i) for i in range(cmap.N)]
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            bounds = np.arange(0, 1.0, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        else:
            # define the colormap
            cmap = plt.get_cmap('bwr')
            # extract all colors from the .jet map
            cmaplist = [cmap(i) for i in range(cmap.N)]
            # create the new map
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            # define the bins and normalize and forcing 0 to be part of the colorbar
            # define the min and max to be -1 and +1 respectively
            bounds = np.arange(-0.5, 0.5, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        fig.colorbar(im, cax=cax, orientation='vertical')

def scale(img):
    # for i in range(img.shape[0]):
    #     for j in range(img.shape[1]):
    #         img[i][j] = ((img[i][j]+1)/2)*255
    return img

def compare_simi(pred, ref):
    return calc_ssim(pred, ref)

In [ ]:
def visualise0(ref):
    """
    Function to visualise the ref imgs

    Args:
        ref (ndarray): the reference data
    """
    # visualise the ref imgs
    ref0 = ref[26, :, :]
    ref1 = ref[:, 30, :]
    ref2 = ref[:, :, 16]
    show_slices([ref0, ref1, ref2])
    plt.suptitle("Center slices for reference image")


def visualise1(ref_ndi, retained_vol, subject, model, layer, affine):
    """
    Function to visualise the NDI imgs and store the difference map

    Args:
        ref_ndi (ndarray): the reference NDI data
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
        retained_vol (int): the number of volumes used
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refNDI0 = ref_ndi[26, :, :]
    refNDI1 = ref_ndi[:, 30, :]
    refNDI2 = ref_ndi[:, :, 16]

    # visualise the pred imgs produced at varied input size
    ndi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI.nii'
    ndi_img = nib.load(ndi_path)
    ndi_data = ndi_img.get_fdata()

    ndi0 = ndi_data[26, :, :]
    ndi1 = ndi_data[:, 30, :]
    ndi2 = ndi_data[:, :, 16]
    show_slices([ndi0, ndi1, ndi2])
    plt.suptitle('Center slices for NDI predicted image by '+model+', input size='+str(retained_vol))
    (score, ndidiff) = compare_ssim(ndi_data, ref_ndi, full=True)
    print(str(retained_vol)+'input size the ssim score for ndi is: ' + str(score))

    # plot the difference map between the imgs by the lib
    ndidiff0 = scale(refNDI0 - ndi0)
    ndidiff1 = scale(refNDI1 - ndi1)
    ndidiff2 = scale(refNDI2 - ndi2)
    show_slices([ndidiff0, ndidiff1, ndidiff2], grayscale=False)
    plt.suptitle("Difference map NDI")

    diff_img_np = ref_ndi - ndi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI_difference.nii')

def visualise2(ref_odi, retained_vol, subject, model, layer, affine):
    """
    Function to visualise the ODI imgs and difference maps

    Args:
        ref_odi (ndarray): the reference ODI data
        retained_vol (int): the number of volumes retained after rejection
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refODI0 = ref_odi[26, :, :]
    refODI1 = ref_odi[:, 30, :]
    refODI2 = ref_odi[:, :, 16]

    # visualise the pred imgs produced at varied input size
    odi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI.nii'
    odi_img = nib.load(odi_path)
    odi_data = odi_img.get_fdata()

    odi0 = odi_data[26, :, :]
    odi1 = odi_data[:, 30, :]
    odi2 = odi_data[:, :, 16]
    show_slices([odi0, odi1, odi2])
    plt.suptitle('Center slices for ODI predicted image by '+model+', input size='+str(retained_vol))
    (score, odidiff) = compare_ssim(odi_data, ref_odi, full=True)
    print(str(retained_vol)+'input size the ssim score for odi is: ' + str(score))

    odidiff0 = scale(refODI0 - odi0)
    odidiff1 = scale(refODI1 - odi1)
    odidiff2 = scale(refODI2 - odi2)
    show_slices([odidiff0, odidiff1, odidiff2], grayscale=False)
    plt.suptitle("Difference map ODI")

    diff_img_np = ref_odi - odi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI_difference.nii')

def visualise3(ref_fwf, retained_vol, subject, model, layer, affine):
    """
    Function to visualise the fwf imgs and difference maps

    Args:
        ref_fwf (ndarray): the reference FWF data
        retained_vol (int): the number of volumes retained after rejection
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refFWF0 = ref_fwf[26, :, :]
    refFWF1 = ref_fwf[:, 30, :]
    refFWF2 = ref_fwf[:, :, 16]

    # visualise the pred imgs produced at varied input size
    fwf_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF.nii'
    fwf_img = nib.load(fwf_path)
    fwf_data = fwf_img.get_fdata()

    fwf0 = fwf_data[26, :, :]
    fwf1 = fwf_data[:, 30, :]
    fwf2 = fwf_data[:, :, 16]
    show_slices([fwf0, fwf1, fwf2])
    plt.suptitle('Center slices for FWF predicted image by '+model+', input size='+str(retained_vol))
    (score, fwfdiff) = compare_ssim(fwf_data, ref_fwf, full=True)
    print(str(retained_vol)+'input size the ssim score for fwf is: ' + str(score))

    # plot the difference map between the imgs by the lib
    fwfdiff0 = scale(refFWF0 - fwf0)
    fwfdiff1 = scale(refFWF1 - fwf1)
    fwfdiff2 = scale(refFWF2 - fwf2)
    show_slices([fwfdiff0, fwfdiff1, fwfdiff2], grayscale=False)
    plt.suptitle("Difference map FWF")

    diff_img_np = ref_fwf - fwf_data
    diff_img = nib.Nifti1Image(diff_img_np, affine)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF_difference.nii')

def visualise4(ref_ndi, ref_odi, ref_fwf, retained_vol, subject, model, layer, affine1, affine2, affine3):
    """
    Function to visualise the imgs and difference maps

    Args:
        ref_ndi (ndarray): the reference NDI data
        ref_odi (ndarray): the reference ODI data
        ref_fwf (ndarray): the reference FWF data
        retained_vol (int): the number of volumes retained after rejection
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refNDI0 = ref_ndi[26, :, :]
    refNDI1 = ref_ndi[:, 30, :]
    refNDI2 = ref_ndi[:, :, 16]

    refODI0 = ref_odi[26, :, :]
    refODI1 = ref_odi[:, 30, :]
    refODI2 = ref_odi[:, :, 16]

    refFWF0 = ref_fwf[26, :, :]
    refFWF1 = ref_fwf[:, 30, :]
    refFWF2 = ref_fwf[:, :, 16]

    # visualise the pred imgs produced at varied input size
    ndi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI.nii'
    odi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI.nii'
    fwf_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF.nii'
    ndi_img = nib.load(ndi_path)
    ndi_data = ndi_img.get_fdata()
    odi_img = nib.load(odi_path)
    odi_data = odi_img.get_fdata()
    fwf_img = nib.load(fwf_path)
    fwf_data = fwf_img.get_fdata()

    ndi0 = ndi_data[26, :, :]
    ndi1 = ndi_data[:, 30, :]
    ndi2 = ndi_data[:, :, 16]
    show_slices([ndi0, ndi1, ndi2])
    plt.suptitle('Center slices for NDI predicted image by '+model+', input size='+str(retained_vol))
    (score, ndidiff) = compare_ssim(ndi_data, ref_ndi, full=True)
    print(str(retained_vol)+'input size the ssim score for ndi is: ' + str(score))

    odi0 = odi_data[26, :, :]
    odi1 = odi_data[:, 30, :]
    odi2 = odi_data[:, :, 16]
    show_slices([odi0, odi1, odi2])
    plt.suptitle('Center slices for ODI predicted image by '+model+', input size='+str(retained_vol))
    (score, odidiff) = compare_ssim(odi_data, ref_odi, full=True)
    print(str(retained_vol)+'input size the ssim score for odi is: ' + str(score))

    fwf0 = fwf_data[26, :, :]
    fwf1 = fwf_data[:, 30, :]
    fwf2 = fwf_data[:, :, 16]
    show_slices([fwf0, fwf1, fwf2])
    plt.suptitle('Center slices for FWF predicted image by '+model+', input size='+str(retained_vol))
    (score, fwfdiff) = compare_ssim(fwf_data, ref_fwf, full=True)
    print(str(retained_vol)+'input size the ssim score for fwf is: ' + str(score))

    # plot the difference map between the imgs by the lib
    ndidiff0 = scale(refNDI0 - ndi0)
    ndidiff1 = scale(refNDI1 - ndi1)
    ndidiff2 = scale(refNDI2 - ndi2)
    show_slices([ndidiff0, ndidiff1, ndidiff2], grayscale=False)
    plt.suptitle("Difference map NDI")

    diff_img_np = ref_ndi - ndi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine1)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI_difference.nii')

    odidiff0 = scale(refODI0 - odi0)
    odidiff1 = scale(refODI1 - odi1)
    odidiff2 = scale(refODI2 - odi2)
    show_slices([odidiff0, odidiff1, odidiff2], grayscale=False)
    plt.suptitle("Difference map ODI")

    diff_img_np = ref_odi - odi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine2)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI_difference.nii')

    fwfdiff0 = scale(refFWF0 - fwf0)
    fwfdiff1 = scale(refFWF1 - fwf1)
    fwfdiff2 = scale(refFWF2 - fwf2)
    show_slices([fwfdiff0, fwfdiff1, fwfdiff2], grayscale=False)
    plt.suptitle("Difference map FWF")

    diff_img_np = ref_fwf - fwf_data
    diff_img = nib.Nifti1Image(diff_img_np, affine3)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF_difference.nii')

___
<h3>Data Preprocessing</h3>

In [ ]:
# motion-free subject path
s01_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/'
s02_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/'
s03_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/'
s04_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/'
# motion-free target labels
s01_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_NDI.nii'
s02_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_NDI.nii'
s03_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/s03_still_reg_NDI.nii'
s04_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/s04_still_reg_NDI.nii'

s01_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_ODI.nii'
s02_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_ODI.nii'
s03_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/s03_still_reg_ODI.nii'
s04_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/s04_still_reg_ODI.nii'

s01_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_FWF.nii'
s02_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_FWF.nii'
s03_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/s03_still_reg_FWF.nii'
s04_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/s04_still_reg_FWF.nii'

In [ ]:
def filter_mask(subpath, fwfpath, threshold=0.99):
    """
    By looking at the imgs generated, we have found out there are some regions that should not be included. Since they have values higher than 1.0
    And we have found out voxels have NDI and ODI values, while that voxel has GROUND TRUTH FWF 1.0
    This should indicate that that voxel should not even be included in the training
    Therefore we want to filter the each subject's mask first, by using their corresponding GROUND TRUTH FWF

    Args:
        subpath (string): the path of the subject folder
        fwfpath (string): the path of the corresponding fwf file
        threshold (float): the thresholds to be used to filter of the mask,
                           a stringnent threshold would be 0.9, the least stringnent threshold is 1.0
                           by default, it is set to 0.99
    """
    # fetch the mask data
    img_mask = nib.load(subpath+'mask-e.nii')
    original_mask = img_mask.get_fdata()
    original_affine = img_mask.affine
    shape = original_mask.shape # retain the shape of the mask
    origin_nonzeros = np.count_nonzero(original_mask)
    print('original mask has: ' + str(origin_nonzeros) + ' of nonzero voxels')
    # fetch the FWF data
    fwf = nib.load(fwfpath).get_fdata()
    # filter
    mask = original_mask.flatten() # this makes a copy of the orginal mask
    fwf = fwf.reshape(mask.shape[0]) # reshape fwf to the corresponding shape
    for i in range(len(mask)):
        # if fwf has high value, means there is no tissue
        # therefore, the voxel should be excluded
        if fwf[i] >= threshold:
            mask[i] = 0.0
    # reshape mask back
    mask = mask.reshape(shape)
    filter_nonzeros = np.count_nonzero(mask)
    print('filtered mask has: ' +str(filter_nonzeros) + ' of nonzero voxels')
    # save the mask
    filter_img = nib.Nifti1Image(mask, original_affine)
    nib.save(filter_img, subpath+'filtered_mask.nii')

In [ ]:
# Use the above code to filter each subject's mask. Store as filtered_mask.nii in each subject folder
filter_mask(s01_path, s01_FWF_path)
filter_mask(s02_path, s02_FWF_path)
filter_mask(s03_path, s04_FWF_path)
filter_mask(s04_path, s03_FWF_path)

In [ ]:
# filtered mask path for each subject
s01_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/filtered_mask.nii'
s02_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/filtered_mask.nii'
s03_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/filtered_mask.nii'
s04_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/filtered_mask.nii'

In [ ]:
# A simple test to verify the algorithm mask_nii_data from utils.nii_utils correctly generates the 1x1 patch from (a) given volume(s) DWI
s02_mask_data = nib.load(s02_mask_path).get_fdata()
print(s02_mask_data.shape)
# generate a random mask
random_mask = np.zeros((s02_mask_data.shape[0],s02_mask_data.shape[1], s02_mask_data.shape[2]))
print(random_mask.shape)
# select some pixels' values in the random_mask turn into 1
# randomly select 4 paris of x, y value
for i in range(4):
    rand_x = np.random.randint(low=0, high=s02_mask_data.shape[0])
    rand_y = np.random.randint(low=0, high=s02_mask_data.shape[1])
    rand_z = np.random.randint(low=0, high=s02_mask_data.shape[2])
    # set the corresponding pixel to have value 1
    random_mask[rand_x][rand_y][rand_z] = 1
# by this, we know the ground truth for this test. The number of 1s in the mask = the number of 1x1 pacth generated from the algorithm
# we apply the random mask onto the s02_mask
data = mask_nii_data(data=s02_mask_data, mask=random_mask)

<strong>Interpret the result from the algorithm test.</strong>
The s02_mask has only one volume of shape (84,84,50). The random mask has a shape (84,84,50).</br>
We randomly turned 4 pixels in the random mask to 1. Then apply the random mask to the s02_mask.</br>
<strong>What we expect to see, there should be 4 patches generated for each volume of s02_mask.</strong></br>
Results show that the data afer masking (by the random mask) has a shape (4,1). Where 4 represents the number of 1x1 patches and 1 represent the volume of s02_mask.</br>
Therefore, it suggests the algorithm works fine.

In [ ]:
"""
Generate base datasets for each subject
"""
ltype = ['N','O','F','A']
for l in ltype:
    print('Generating basedataset for label: ' + l)
    cmd = '--base --label_type ' + l + ' --subjects s01_still s02_still s03_still_reg s04_still_reg'
    args = data_parser().parse_args(cmd.split())
    generate_data(args)

In [ ]:
"""
Using nib to fetch the ground truth img
"""
#load the truth data for subject 1
s01_NDI_img = nib.load(s01_NDI_path)
s01_ODI_img = nib.load(s01_ODI_path)
s01_FWF_img = nib.load(s01_FWF_path)
s01_NDI_affine = s01_NDI_img.affine
s01_ODI_affine = s01_ODI_img.affine
s01_FWF_affine = s01_FWF_img.affine
s01_NDI_img_data = s01_NDI_img.get_fdata()
s01_ODI_img_data = s01_ODI_img.get_fdata()
s01_FWF_img_data = s01_FWF_img.get_fdata()
#load the truth data for subject 2
s02_NDI_img = nib.load(s02_NDI_path)
s02_ODI_img = nib.load(s02_ODI_path)
s02_FWF_img = nib.load(s02_FWF_path)
s02_NDI_affine = s02_NDI_img.affine
s02_ODI_affine = s02_ODI_img.affine
s02_FWF_affine = s02_FWF_img.affine
s02_NDI_img_data = s02_NDI_img.get_fdata()
s02_ODI_img_data = s02_ODI_img.get_fdata()
s02_FWF_img_data = s02_FWF_img.get_fdata()
# load the truth data for subject 3
s03_NDI_img = nib.load(s03_NDI_path)
s03_ODI_img = nib.load(s03_ODI_path)
s03_FWF_img = nib.load(s03_FWF_path)
s03_NDI_affine = s03_NDI_img.affine
s03_ODI_affine = s03_ODI_img.affine
s03_FWF_affine = s03_FWF_img.affine
s03_NDI_img_data = s03_NDI_img.get_fdata()
s03_ODI_img_data = s03_ODI_img.get_fdata()
s03_FWF_img_data = s03_FWF_img.get_fdata()
# load the truth data for subject 4
s04_NDI_img = nib.load(s04_NDI_path)
s04_ODI_img = nib.load(s04_ODI_path)
s04_FWF_img = nib.load(s04_FWF_path)
s04_NDI_affine = s04_NDI_img.affine
s04_ODI_affine = s04_ODI_img.affine
s04_FWF_affine = s04_FWF_img.affine
s04_NDI_img_data = s04_NDI_img.get_fdata()
s04_ODI_img_data = s04_ODI_img.get_fdata()
s04_FWF_img_data = s04_FWF_img.get_fdata()

___

<strong>NDI</strong>

In [ ]:
"""
Generate training dataset NDI for ANN.
"""
cmd = "--subjects s01_still s02_still s03_still_reg s04_still_reg --fc1d --label_type N"
args = data_parser().parse_args(cmd.split())
generate_data(args)

<strong>
Interpret this result.
</strong> 
After the mask is applied to 96 volumes of DWI imgs, for each volume of img, there are 81882 1x1 patches (i.e. voxels) generated. Therefore, in total there are 81882x96=7860672 values as training dataset. <br/>
For each masked voxel, there is a corresponding NDI value. Therefore, the number of NDI values is the same as the number of masked voxels (82889). Becasue we are examine NDI only, hence only one value associated with each masked voxel.

In [ ]:
# Generate the dataset for 2D CNN
cmd = "--subjects s01_still s02_still s03_still_reg s04_still_reg --conv2d --label_type N"
args = data_parser().parse_args(cmd.split())
generate_data(args)

<strong>
Interpret the result
</strong>
The masked voxels are 81634. For each masked voxel, there is a 3x3 patch which includes this masked voxel at the centre of the patch; but also including its 8 neighbouring voxels. Each masked voxel has 96 volumes of the 3x3 patch. Therefore, in total there are 81634 x 3 x 3 x 96 = 70531776 values as the training dataset.<br/>
Herein, because we only examine NDI. For each masked voxel, there is a corresponded predicted NDI value. Hence the number of masked voxels is the same for label and data. The label size is 1x1 because it is the NDI parameter for the masked voxel. The masked voxel has a size 1x1. And there is only one of this value for each masked voxel, since only NDI is considered.

In [ ]:
# Generate the dataset for 3D CNN
cmd = "--subjects s01_still s02_still s03_still_reg s04_still_reg --conv3d --label_type N"
args = data_parser().parse_args(cmd.split())
generate_data(args)

<strong>
Interpret the result
</strong>
The masked voxels are 81634. For each masked voxel, there is a 3x3x3 patch which includes this masked voxel at the centre of the patch; but also including its 26 neighbouring voxels. Each masked voxel has 96 volumes of 3x3x3 patch. Therefore, in total there are 81634 x 3 x 3 x 3 x 96 = 211595328 values as the training dataset.<br/>
Herein, because we only examine NDI. For each masked voxel, there is a corresponded predicted NDI value. Hence the number of masked voxels is the same for label and data. The label size is 1x1x1 because it is the NDI parameter for the masked voxel. The masked voxel has a size 1x1x1. And there is only one of this value for each masked voxel, since only NDI is considered.

<h4>Training</h4>

<strong>
The split between validation dataset and the training dataset is 5:5<br/>
The learning rate is 0.0001 to ensure a smooth learning<br/>
Herein, at most 6 hidden fully connected layers are trained.<br/>
Loss curve for each training is plotted.<br/>
The test result is also generated -- evaluating the RMSE and SSIM between pred and reference.<br/>
The reference image and the genereated image are displayed side by side.</br>
The difference map between the ref img and the pred img is also generated
</strong>

<strong>1 layer</strong>

In [ ]:
layer = 1

ANN

In [ ]:
# 1 layer training
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type N'
plot_loss(cmd)

In [ ]:
# 1 layer testing
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
# 1 layer training
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type N'
plot_loss(cmd)

In [ ]:
# 1 layer testing
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
# 1 layer training
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type N'
plot_loss(cmd)

In [ ]:
# 1 layer testing
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_NDI_img_data)
visualise1(s02_NDI_img_data, 96, 's02_still', 'fc1d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv2d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv3d',layer, s02_NDI_affine)

<strong>2 layer</strong>

In [ ]:
layer = 2 

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_NDI_img_data)
visualise1(s02_NDI_img_data, 96, 's02_still', 'fc1d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv2d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv3d',layer, s02_NDI_affine)

<strong>3 layer</strong>

In [ ]:
layer = 3

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_NDI_img_data)
visualise1(s02_NDI_img_data, 96, 's02_still', 'fc1d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv2d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv3d',layer, s02_NDI_affine)

<strong>4 layer</strong>

In [ ]:
layer = 4

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_NDI_img_data)
visualise1(s02_NDI_img_data, 96, 's02_still', 'fc1d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv2d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv3d',layer, s02_NDI_affine)

<strong>5 layer</strong>

In [ ]:
layer = 5

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_NDI_img_data)
visualise1(s02_NDI_img_data, 96, 's02_still', 'fc1d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv2d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv3d',layer, s02_NDI_affine)

<strong>6 layer</strong>

In [ ]:
layer = 6

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type N'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type N'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_NDI_img_data)
visualise1(s02_NDI_img_data, 96, 's02_still', 'fc1d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv2d',layer, s02_NDI_affine)
visualise1(s02_NDI_img_data, 96, 's02_still', 'conv3d',layer, s02_NDI_affine)

___

<strong>ODI</strong>


In [ ]:
"""
Generate training dataset ODI for ANN.
"""
cmd = "--subjects s01_still s02_still s03_still_reg s04_still_reg --fc1d --label_type O"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Generate training dataset ODI for 2D CNN.
"""
cmd = "--subjects s01_still s02_still s03_still_reg s04_still_reg --conv2d --label_type O"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Generate training dataset ODI for 3D CNN.
"""
cmd = "--subjects s01_still s02_still s03_still_reg s04_still_reg --conv3d --label_type O"
args = data_parser().parse_args(cmd.split())
generate_data(args)

<strong>1 layer</strong>

In [ ]:
layer = 1

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_ODI_img_data)
visualise2(s02_ODI_img_data, 96, 's02_still', 'fc1d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv2d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv3d',layer, s02_ODI_affine)

<strong>2 layer</strong>

In [ ]:
layer = 2 

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_ODI_img_data)
visualise2(s02_ODI_img_data, 96, 's02_still', 'fc1d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv2d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv3d',layer, s02_ODI_affine)

<strong>3 layer</strong>

In [ ]:
layer = 3

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_ODI_img_data)
visualise2(s02_ODI_img_data, 96, 's02_still', 'fc1d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv2d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv3d',layer, s02_ODI_affine)

<strong>4 layer</strong>

In [ ]:
layer = 4

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_ODI_img_data)
visualise2(s02_ODI_img_data, 96, 's02_still', 'fc1d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv2d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv3d',layer, s02_ODI_affine)

<strong>5 layer</strong>

In [ ]:
layer = 5

<strong>ANN</strong>

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

<strong>2D CNN</strong>

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

<strong>3D CNN</strong>

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_ODI_img_data)
visualise2(s02_ODI_img_data, 96, 's02_still', 'fc1d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv2d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv3d',layer, s02_ODI_affine)

<strong>6 layer</strong>

In [ ]:
layer = 6

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type O'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type O'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_ODI_img_data)
visualise2(s02_ODI_img_data, 96, 's02_still', 'fc1d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv2d',layer, s02_ODI_affine)
visualise2(s02_ODI_img_data, 96, 's02_still', 'conv3d',layer, s02_ODI_affine)

<strong>FWF</strong>

In [ ]:
# Generate the dataset for ANNc
cmd = "--subjects s01_still s02_still s03_still_reg s04_still_reg --fc1d --label_type F"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
# Generate the dataset for 2D CNN
cmd = "--subjects s01_still s02_still s03_still_reg s04_still_reg --conv2d --label_type F"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
# Generate the dataset for 3D CNN
cmd = "--subjects s01_still s02_still s03_still_reg s04_still_reg --conv3d --label_type F"
args = data_parser().parse_args(cmd.split())
generate_data(args)

<strong>1 layer</strong>

In [ ]:
layer = 1

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_FWF_img_data)
visualise3(s02_FWF_img_data, 96, 's02_still', 'fc1d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv2d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv3d',layer, s02_FWF_affine)

<strong>2 layer</strong>

In [ ]:
layer = 2

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_FWF_img_data)
visualise3(s02_FWF_img_data, 96, 's02_still', 'fc1d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv2d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv3d',layer, s02_FWF_affine)

<strong>3 layer</strong>

In [ ]:
layer = 3 

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_FWF_img_data)
visualise3(s02_FWF_img_data, 96, 's02_still', 'fc1d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv2d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv3d',layer, s02_FWF_affine)

<strong>4 layer</strong>

In [ ]:
layer = 4

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_FWF_img_data)
visualise3(s02_FWF_img_data, 96, 's02_still', 'fc1d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv2d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv3d',layer, s02_FWF_affine)

<strong>5 layer</strong>

In [ ]:
layer = 5

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_FWF_img_data)
visualise3(s02_FWF_img_data, 96, 's02_still', 'fc1d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv2d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv3d',layer, s02_FWF_affine)

<strong>6 layer</strong>

In [ ]:
layer = 6

ANN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model fc1d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

2D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv2d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

3D CNN

In [ ]:
cmd = '--train_subjects '+train_subject+' --model conv3d --layer '+layer+' --train --label_type F'
plot_loss(cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer '+layer+' --label_type F'
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# load the reference img and pred img
visualise0(s02_FWF_img_data)
visualise3(s02_FWF_img_data, 96, 's02_still', 'fc1d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv2d',layer, s02_FWF_affine)
visualise3(s02_FWF_img_data, 96, 's02_still', 'conv3d',layer, s02_FWF_affine)

___

<h4>Test the performace of each network to generate each parameter with varied number of DWI as input size</h4>

Each network (ANN, 2D CNN and 3D CNN) should be implemented with 4 hidden layers. The choice is suggested from the obtained results from the previous experiment

<h4>Training</h4><br/>
ANN

In [ ]:
# define the list of parameters required to be trained and tested
params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# train
for param in params:
    # vary the input size; in this case becasue I know the max input size is 96
    for i in np.arange(start=6, stop=97, step=10):
        # specify the param to be trained and the size of the input
        train_cmd = "--train_subjects s01_still --model fc1d --layer 4 --train --label_type " + param[0] + " --DWI " + str(i)
        # for each training, plot the corresponding loss graph
        plot_loss(train_cmd)

In [ ]:
# define the list of parameters required to be trained and tested
params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# test
for param in params:
    # vary the input size; in this case becasue I know the max input size is 96
    for i in np.arange(start=6, stop=97, step=10):
        # specify the param to be tested and the size of the test data
        print(i)
        test_cmd = "--test_subjects s02_still --model fc1d --layer 4 --label_type " + param[0] + " --DWI " + str(i)
        args = model_parser().parse_args(test_cmd.split())
        # test the trained model
        test_model(args) 

In [ ]:
# plot each derived image
# specify the parameters
params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
ref_img = [s02_NDI_img, s02_ODI_img, s02_FWF_img] # this is the ref img data list
ref_img_data = [s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data] # this is the ref img data list
# iterate, so plot each param in order
for i in range (len(params)):
    # plot the reference img first
    ref_slice_0 = ref_img_data[i][26, :, :]
    ref_slice_1 = ref_img_data[i][:, 30, :]
    ref_slice_2 = ref_img_data[i][:, :, 16]
    show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
    plt.suptitle("Center slices for s02_still_" + params[i] + " reference image")
    
    affine = ref_img[i].affine
    
    for j in np.arange(start=6, stop=97, step=10):
        # plot the predicted imgs for different input size
        pred_path = '../Net/nii/s02_still-' + str(j)+'-fc1d-patch_1-base_1-layer_4-label_'+params[i]+'.nii'
        # load the img
        pred_img = nib.load(pred_path)
        # fetch the data
        pred_data = pred_img.get_fdata()

        # plot the pred img
        pred0 = pred_data[26, :, :]
        pred1 = pred_data[:, 30, :]
        pred2 = pred_data[:, :, 16]
        show_slices([pred0, pred1, pred2])
        plt.suptitle("Center slices for s02_still_" + params[i] + " predicted image by ANN, input size="+str(j))

        # plot the difference img
        diff0 = ref_slice_0 - pred0
        diff1 = ref_slice_1 - pred1
        diff2 = ref_slice_2 - pred2
        show_slices([diff0, diff1, diff2], grayscale=False)
        plt.suptitle('Difference map')

        diff_img_np = ref_img_data[i] - pred_data
        diff_img = nib.Nifti1Image(diff_img_np, affine)
        nib.save(diff_img, '../Net/nii/s02_still-' + str(j)+'-fc1d-patch_1-base_1-layer_4-label_'+params[i]+'_difference.nii')   

5 Layer


In [ ]:
# # define the list of parameters required to be trained and tested
# params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# # train
# for param in params:
#     # vary the input size; in this case becasue I know the max input size is 96
#     for i in np.arange(start=6, stop=97, step=10):
#         # specify the param to be trained and the size of the input
#         train_cmd = "--train_subjects s01_still --model fc1d --layer 5 --train --label_type " + param[0] + " --DWI " + str(i)
#         # for each training, plot the corresponding loss graph
#         plot_loss(train_cmd)

In [ ]:
# # define the list of parameters required to be trained and tested
# params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# # test
# for param in params:
#     # vary the input size; in this case becasue I know the max input size is 96
#     for i in np.arange(start=6, stop=97, step=10):
#         # specify the param to be tested and the size of the test data
#         print(i)
#         test_cmd = "--test_subjects s02_still --model fc1d --layer 5 --label_type " + param[0] + " --DWI " + str(i)
#         args = model_parser().parse_args(test_cmd.split())
#         # test the trained model
#         test_model(args) 

2D CNN<br/>
repeat exactly the same process as above, but change the network to 2D CNN

In [ ]:
# define the list of parameters required to be trained and tested
params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# train
for param in params:
    # vary the input size; in this case becasue I know the max input size is 96
    for i in np.arange(start=6, stop=97, step=10):
        # specify the param to be trained and the size of the input
        train_cmd = "--train_subjects s01_still --model conv2d --layer 4 --train --label_type " + param[0] + " --DWI " + str(i)
        # for each training, plot the corresponding loss graph
        plot_loss(train_cmd)

In [ ]:
# define the list of parameters required to be trained and tested
params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# test
for param in params:
    # vary the input size; in this case becasue I know the max input size is 96
    for i in np.arange(start=6, stop=97, step=10):
        # specify the param to be tested and the size of the test data
        test_cmd = "--test_subjects s02_still --model conv2d --layer 4 --label_type " + param[0] + " --DWI " + str(i)
        args = model_parser().parse_args(test_cmd.split())
        # test the trained model
        test_model(args)

In [ ]:
# plot each derived image
# specify the parameters
params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
ref_img = [s02_NDI_img, s02_ODI_img, s02_FWF_img] # this is the ref img data list
ref_img_data = [s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data] # this is the ref img data list
# iterate, so plot each param in order
for i in range (len(params)):
    # plot the reference img first
    ref_slice_0 = ref_img_data[i][26, :, :]
    ref_slice_1 = ref_img_data[i][:, 30, :]
    ref_slice_2 = ref_img_data[i][:, :, 16]
    show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
    plt.suptitle("Center slices for s02_still_" + params[i] + " reference image")
    
    affine = ref_img[i].affine
    
    for j in np.arange(start=6, stop=97, step=10):
        # plot the predicted imgs for different input size
        pred_path = '../Net/nii/s02_still-' + str(j)+'-conv2d-patch_3-base_1-layer_4-label_'+params[i]+'.nii'
        # load the img
        pred_img = nib.load(pred_path)
        # fetch the data
        pred_data = pred_img.get_fdata()

        # plot the pred img
        pred0 = pred_data[26, :, :]
        pred1 = pred_data[:, 30, :]
        pred2 = pred_data[:, :, 16]
        show_slices([pred0, pred1, pred2])
        plt.suptitle("Center slices for s02_still_" + params[i] + " predicted image by 2D CNN, input size="+str(j))

        # plot the difference img
        diff0 = ref_slice_0 - pred0
        diff1 = ref_slice_1 - pred1
        diff2 = ref_slice_2 - pred2
        show_slices([diff0, diff1, diff2], grayscale=False)
        plt.suptitle('Difference map')

        diff_img_np = ref_img_data[i] - pred_data
        diff_img = nib.Nifti1Image(diff_img_np, affine)
        nib.save(diff_img, '../Net/nii/s02_still-' + str(j)+'-conv2d-patch_3-base_1-layer_4-label_'+params[i]+'_difference.nii')   

5 Layer

In [ ]:
# # define the list of parameters required to be trained and tested
# params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# # train
# for param in params:
#     # vary the input size; in this case becasue I know the max input size is 96
#     for i in np.arange(start=6, stop=97, step=10):
#         # specify the param to be trained and the size of the input
#         train_cmd = "--train_subjects s01_still --model conv2d --layer 5 --train --label_type " + param[0] + " --DWI " + str(i)
#         # for each training, plot the corresponding loss graph
#         plot_loss(train_cmd)

In [ ]:
# # define the list of parameters required to be trained and tested
# params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# # test
# for param in params:
#     # vary the input size; in this case becasue I know the max input size is 96
#     for i in np.arange(start=6, stop=97, step=10):
#         # specify the param to be tested and the size of the test data
#         test_cmd = "--test_subjects s02_still --model conv2d --layer 5 --label_type " + param[0] + " --DWI " + str(i)
#         args = model_parser().parse_args(test_cmd.split())
#         # test the trained model
#         test_model(args)

3D CNN<br/>
repeat the same process as above

In [ ]:
# define the list of parameters required to be trained and tested
params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# train
for param in params:
    # vary the input size; in this case becasue I know the max input size is 96
    for i in np.arange(start=6, stop=97, step=10):
        # specify the param to be trained and the size of the input
        train_cmd = "--train_subjects s01_still --model conv3d --layer 4 --train --label_type " + param[0] + " --DWI " + str(i)
        # for each training, plot the corresponding loss graph
        plot_loss(train_cmd)

In [ ]:
# define the list of parameters required to be trained and tested
params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# test
for param in params:
    # vary the input size; in this case becasue I know the max input size is 96
    for i in np.arange(start=6, stop=97, step=10):
        # specify the param to be tested and the size of the test data
        test_cmd = "--test_subjects s02_still --model conv3d --layer 4 --label_type " + param[0] + " --DWI " + str(i)
        args = model_parser().parse_args(test_cmd.split())
        # test the trained model
        test_model(args)

In [ ]:
# plot each derived image
# specify the parameters
params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
ref_img = [s02_NDI_img, s02_ODI_img, s02_FWF_img] # this is the ref img data list
ref_img_data = [s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data] # this is the ref img data list
# iterate, so plot each param in order
for i in range (len(params)):
    # plot the reference img first
    ref_slice_0 = ref_img_data[i][26, :, :]
    ref_slice_1 = ref_img_data[i][:, 30, :]
    ref_slice_2 = ref_img_data[i][:, :, 16]
    show_slices([ref_slice_0, ref_slice_1, ref_slice_2])
    plt.suptitle("Center slices for s02_still_" + params[i] + " reference image")
    
    affine = ref_img[i].affine
    
    for j in np.arange(start=6, stop=97, step=10):
        # plot the predicted imgs for different input size
        pred_path = '../Net/nii/s02_still-' + str(j)+'-conv3d-patch_3-base_1-layer_4-label_'+params[i]+'.nii'
        # load the img
        pred_img = nib.load(pred_path)
        # fetch the data
        pred_data = pred_img.get_fdata()

        # plot the pred img
        pred0 = pred_data[26, :, :]
        pred1 = pred_data[:, 30, :]
        pred2 = pred_data[:, :, 16]
        show_slices([pred0, pred1, pred2])
        plt.suptitle("Center slices for s02_still_" + params[i] + " predicted image by 3D CNN, input size="+str(j))

        # plot the difference img
        diff0 = ref_slice_0 - pred0
        diff1 = ref_slice_1 - pred1
        diff2 = ref_slice_2 - pred2
        show_slices([diff0, diff1, diff2], grayscale=False)
        plt.suptitle('Difference map')

        diff_img_np = ref_img_data[i] - pred_data
        diff_img = nib.Nifti1Image(diff_img_np, affine)
        nib.save(diff_img, '../Net/nii/s02_still-' + str(j)+'-conv3d-patch_1-base_1-layer_4-label_'+params[i]+'_difference.nii')   

5 Layer

In [ ]:
# # define the list of parameters required to be trained and tested
# params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# # train
# for param in params:
#     # vary the input size; in this case becasue I know the max input size is 96
#     for i in np.arange(start=6, stop=97, step=10):
#         # specify the param to be trained and the size of the input
#         train_cmd = "--train_subjects s01_still --model conv3d --layer 5 --train --label_type " + param[0] + " --DWI " + str(i)
#         # for each training, plot the corresponding loss graph
#         plot_loss(train_cmd)

In [ ]:
# # define the list of parameters required to be trained and tested
# params = ['NDI', 'ODI', 'FWF'] # this is for NODDI
# # test
# for param in params:
#     # vary the input size; in this case becasue I know the max input size is 96
#     for i in np.arange(start=6, stop=97, step=10):
#         # specify the param to be tested and the size of the test data
#         test_cmd = "--test_subjects s02_still --model conv3d --layer 5 --label_type " + param[0] + " --DWI " + str(i)
#         args = model_parser().parse_args(test_cmd.split())
#         # test the trained model
#         test_model(args)